In [ ]:
# !pip install tensorflow==1.15.0

In [ ]:
# !pip install tensorflow-gpu==1.15.0

In [ ]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
from collections import Counter
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

In [ ]:
tf.__version__

In [ ]:
random_seed = [1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 10000]

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--dataname", default = "data_-1.5c15", type = str)

parser.add_argument("--distribution", default = "ZINB")
parser.add_argument("--self_training", default = True)
parser.add_argument("--dims", default = [499, 256, 64, 32])
parser.add_argument("--highly_genes", default = 500)
parser.add_argument("--alpha", default = 0.001, type = float)
parser.add_argument("--gamma", default = 0.001, type = float)
parser.add_argument("--learning_rate", default = 0.0001, type = float)
parser.add_argument("--random_seed", default = random_seed)
parser.add_argument("--batch_size", default = 256, type = int)
parser.add_argument("--update_epoch", default = 10, type = int)
parser.add_argument("--pretrain_epoch", default = 1000, type = int)
parser.add_argument("--funetrain_epoch", default = 2000, type = int)
parser.add_argument("--t_alpha", default = 1.0)
parser.add_argument("--noise_sd", default = 1.5)
parser.add_argument("--error", default = 0.001, type = float)
parser.add_argument("--gpu_option", default = "0")

args = parser.parse_args("")

In [ ]:
for category in ["balanced_data", "imbalanced_data", "real_data"
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue

        X = np.ceil(X).astype(np.int)
        count_X = X
        print(X.shape, count_X.shape)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=args.highly_genes,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])

        high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
        count_X = count_X[:, high_variable]
        size_factor = np.array(adata.obs.size_factors).reshape(-1,
                                                               1).astype(np.float32)
        cluster_number = int(max(Y) - min(Y) + 1)
        args.dims[0]=X.shape[1]
        print(X.shape, count_X.shape)

        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
            tf.reset_default_graph()
            chencluster = autoencoder(args.dataname, args.distribution,
                                      args.self_training, args.dims, cluster_number,
                                      args.t_alpha, args.alpha, args.gamma,
                                      args.learning_rate, args.noise_sd)
            chencluster.pretrain(X, count_X, size_factor, args.batch_size,
                                 args.pretrain_epoch, args.gpu_option)

            chencluster.funetrain(X, count_X, size_factor, args.batch_size,
                                  args.funetrain_epoch, args.update_epoch, args.error, Y)

            kmeans_ARI = np.around(adjusted_rand_score(Y, chencluster.kmeans_pred), 5)
            kmeans_NMI = np.around(
                normalized_mutual_info_score(Y, chencluster.kmeans_pred), 5)

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, chencluster.Y_pred)
            NMI = np.around(normalized_mutual_info_score(Y, chencluster.Y_pred), 5)
            ss = silhouette_score(chencluster.latent_repre,chencluster.Y_pred)
            cal = calinski_harabasz_score(chencluster.latent_repre,chencluster.Y_pred)

            df.loc[df.shape[0]] = [dataset, ARI, NMI, ss, run, elapsed, chencluster.Y_pred, cal]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_sczi.pkl")


In [ ]:
df.mean()

In [ ]:
df.groupby("dataset").mean()